In [ ]:
-- uploaded pdfs into stage @AI_SQL_DEMO_DB.PUBLIC.EQUITY_RESEARCH_DEMO
-- (note: please create stage as client side encryption)

CREATE OR REPLACE TABLE raw_docs_text AS SELECT
    relative_path, 
    GET_PRESIGNED_URL(@INSURANCE_WORKSHOP_DB.RAW_DATA.POLICY_WORDING_DOCUMENTS, relative_path) as scoped_file_url, 
    TO_VARIANT(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(@INSURANCE_WORKSHOP_DB.RAW_DATA.POLICY_WORDING_DOCUMENTS, relative_path , {'mode': 'layout'})) as raw_text_dict,
    raw_text_dict:content as raw_text
FROM DIRECTORY(@INSURANCE_WORKSHOP_DB.RAW_DATA.POLICY_WORDING_DOCUMENTS);

select * from raw_docs_text;

In [ ]:
CREATE OR REPLACE TABLE insurance_policy_extractions AS
SELECT
    RELATIVE_PATH,
    raw_extraction_output:company::VARCHAR AS company,
    raw_extraction_output:date::VARCHAR AS date
FROM (
    SELECT
        RELATIVE_PATH,
        AI_COMPLETE(
            model => 'snowflake-arctic',
            -- model => 'claude-4-sonnet',
            -- model => 'claude-3-7-sonnet',
            -- model => 'llama3.3-70b',
            prompt => CONCAT(
                'From the following insurance policy text, extract the name of the insurer (company) and the policy effective date. If a specific day is missing from the date, assume the first day of the month and return in "yyyy-mm".\n\nTEXT:\n', 
                LEFT(RAW_TEXT, 3000)
            ),
            response_format => {
                'type': 'json',
                'schema': {
                    'type': 'object',
                    'properties': {
                        'company': {'type': 'string'},
                        'date': {'type': 'string'}
                    },
                    'required': ['company', 'date']
                }
            }
        ) AS raw_extraction_output
    FROM
        raw_docs_text
) ai_results; 

select * from insurance_policy_extractions;

In [ ]:
CREATE OR REPLACE TABLE insurance_policy_doc_metadata AS
(
SELECT 
    rdt.*,
    ipe.company,
    ipe.date
FROM 
    raw_docs_text rdt
INNER JOIN 
    insurance_policy_extractions ipe
    ON rdt.RELATIVE_PATH = ipe.RELATIVE_PATH); 

select * from insurance_policy_doc_metadata;

In [ ]:
desc table insurance_policy_doc_metadata;

In [ ]:
CREATE OR REPLACE TABLE insurance_docs_chunked AS
SELECT
    RELATIVE_PATH,
    SCOPED_FILE_URL,
    COMPANY,
    DATE,
    ['INSURANCE_DOCS'] AS DOCUMENT_TYPE,
    SPLIT_PART(RELATIVE_PATH, '/', -1)::TEXT AS DOCUMENT,
    VALUE::TEXT AS TEXT
FROM
    (SELECT 
        rdt.*,
        ipe.company,
        ipe.date
    FROM 
        raw_docs_text rdt
    INNER JOIN 
        insurance_policy_extractions ipe
        ON rdt.RELATIVE_PATH = ipe.RELATIVE_PATH) joined_data,
    LATERAL FLATTEN(
        SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER(RAW_TEXT_DICT::TEXT, 
                                                        'markdown', 
                                                        4000, 
                                                        400, 
                                                        ['\n\n', ' '])
    );

SELECT
    *
FROM
    insurance_docs_chunked
LIMIT
    10;

In [ ]:
SELECT
    *
FROM
    insurance_docs_chunked;